In [ ]:
import requests

url = 'https://www.gmarket.co.kr/n/best'

response = requests.get(url)
if response.status_code == 200:
    print('정상')
else:
    print(response.status_code)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, 'html.parser')
ul = soup.select_one('div.best-list > ul')
best100 = ul.select('li')[:100]

In [ ]:
import pymysql
conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                        port = 3306, database = 'python_db')
cursor = conn.cursor()
sql = '''insert into goods values (%s, %s, %s, %s)'''

In [ ]:
for item in best100:
    title = item.select_one('a.itemname').text
    code = item.select_one('div.thumb img').get('src').split('/')[3]
    try:
        o_price = item.select_one('div.o-price span:nth-child(2)').text.replace('원', '').replace(',','')
        s_price = item.select_one('div.s-price strong').text.replace('원', '').replace(',','')
    except:
        o_price = item.select_one('div.s-price strong').text.replace('원', '').replace(',','')
        s_price = o_price
    cursor.execute(sql, [code, title, o_price, s_price])

conn.commit()
    

In [ ]:
conn.close()

In [ ]:
max([len(i) for i in titles])

In [ ]:
# <li class="first">
# <span class="box__corners-id"></span>
# <p class="no1">1</p>
# <div class="thumb">
#     <a aria-hidden="true" href="http://item.gmarket.co.kr/Item?goodscode=2183617402&amp;ver=20240116" tabindex="-1">
#         <img alt="[펩시]펩시콜라 제로슈거 라임 355ml 24캔 캔음료" class="image__lazy" decoding="async" height="210" loading="lazy" src="//gdimg.gmarket.co.kr/2183617402/still/300?ver=1682493949" width="210"/>
#     </a>
# </div>
# <a class="itemname" href="http://item.gmarket.co.kr/Item?goodscode=2183617402&amp;ver=20240116">[펩시]펩시콜라 제로슈거 라임 355ml 24캔 캔음료</a>
# <div class="item_price">
#     <div class="o-price">
#         <span class="for-a11y">정가</span>
#         <span>21,400<!-- -->원</span></div>
#     <div class="s-price">
#         <span class="for-a11y">할인가</span>
#         <strong><span>16,500<!-- -->원</span></strong> 
#         <span><span class="sale"></span><em>22<!-- -->%</em></span>
#     </div>
#     <div class="icon">
#         <img alt="스마일배송" src="//pics.gmarket.co.kr/pc/ko/corner/gbest/ic_smile_shipped.png"/>
#         <img alt="무료배송" src="//pics.gmkt.kr/pc/ko/corner/gbest/ic_freeshipping.gif"/>
#     </div>
# </div>
# </li>

In [ ]:
from datetime import datetime

timestamp = 163141337
date = datetime.utcfromtimestamp(timestamp)

print(date.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                        port = 3306, database = 'python_db')
cursor = conn.cursor()
cursor.execute('''select code from goods''')
codes = cursor.fetchall()
print(codes)

In [ ]:
conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                        port = 3306, database = 'python_db')
cursor = conn.cursor()
sql = '''insert into review (code, title, content) values (%s, %s, %s)'''

In [ ]:
for code in codes:
    url = 'http://item.gmarket.co.kr/Review/Premium'
    for page in range(1,21):
        break_point = 0
        params = {'goodsCode': code[0],
                'pageNo': page,
                'sort': 0,
                'totalPage': 0 }
        
        response = requests.post(url, params)

        if response.status_code == 200:
            # 크롤링(총 리뷰의 수)
            review = BeautifulSoup(response.content, 'html.parser')
            rows = review.select('table > tbody tr')
            for row in rows:
                try:
                    title = row.select_one('td p:nth-child(1)').text.strip()
                    content = row.select_one('td p:nth-child(3)').text.strip()
                except:
                    break_point = 1
                    break
                try:
                    cursor.execute(sql, [code[0], title, content])
                except:
                    pass
            conn.commit()
            if break_point == 1:
                break
        else:
            continue
        
    

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [ ]:
## 쥐마켓 best100 데이터 크롤링
# 1. request
# 2. beautifulsoup
best100 = best100_crawl()
if best100:
    # 3. pymysql
    # 4. 테이블에 저장
    best100_insert(best100)

In [ ]:
## 리뷰 데이터 크롤링
# 1. best100 - code select
codes = code_select()
# 2. request
# 3. beautifulsoup
# 4. 테이블에 저장
review_crawl(codes)

In [ ]:
## 쥐마켓 best100 데이터 크롤링
# 1. request
# 2. beautifulsoup
def best100_crawl():    
    url = 'https://www.gmarket.co.kr/n/best'

    response = requests.get(url)
    print(response.status_code)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ul = soup.select_one('div.best-list > ul')
        best100 = ul.select('li')[:100]
        return best100
    else:
        return []

In [ ]:
# 3. pymysql
# 4. 테이블에 저장
def best100_insert(best100):
    conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                            port = 3306, database = 'python_db')
    cursor = conn.cursor()
    sql = '''insert into goods values (%s, %s, %s, %s)'''
    for item in best100:
        title = item.select_one('a.itemname').text
        code = item.select_one('div.thumb img').get('src').split('/')[3]
        try:
            o_price = item.select_one('div.o-price span:nth-child(2)').text.replace('원', '').replace(',','')
            s_price = item.select_one('div.s-price strong').text.replace('원', '').replace(',','')
        except:
            o_price = item.select_one('div.s-price strong').text.replace('원', '').replace(',','')
            s_price = o_price
        cursor.execute(sql, [code, title, o_price, s_price])

    conn.commit()
    conn.close()

In [ ]:
# 1. best100 - code select
def code_select():
    conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                            port = 3306, database = 'python_db')
    cursor = conn.cursor()
    cursor.execute('''select code from goods''')
    codes = cursor.fetchall()
    conn.close()
    return codes

In [ ]:
# 2. request
# 3. beautifulsoup
# 4. 테이블에 저장
def review_crawl(codes):
    conn = pymysql.connect(user = 'maria', password = '1234', host = 'localhost',
                                port = 3306, database = 'python_db')
    cursor = conn.cursor()
    for code in codes:
        url = 'http://item.gmarket.co.kr/Review/Premium'
        for page in range(1,6):
            break_point = 0
            params = {'goodsCode': code[0],
                    'pageNo': page,
                    'sort': 0,
                    'totalPage': 0 }

            response = requests.post(url, params)

            if response.status_code == 200:
                # 크롤링(총 리뷰의 수)
                review = BeautifulSoup(response.content, 'html.parser')
                rows = review.select('table > tbody tr')
                for row in rows:
                    try:
                        title = row.select_one('td p:nth-child(1)').text.strip()
                        content = row.select_one('td p:nth-child(3)').text.strip()
                    except:
                        break_point = 1
                        break
                    try:
                        cursor.execute(sql, [code[0], title, content])
                    except:
                        pass
                conn.commit()
                if break_point == 1:
                    break
            else:
                continue
    conn.close()